In [2]:
import os
import numpy as np
from PIL import Image

# -----------------------------
# 1. PARAMETERS
# -----------------------------

image_size = (64,64) # Resizes all the loaded images into a size of 64X64 pixels 
data_directory = "Images" # Specifies the root directory containing the image sub-folders (ie. "Dogs" and "Cats")

# -----------------------------
# 2. LOAD IMAGES
# -----------------------------
def loading_images(data_directory,mode, image_size):

    """
    Loads images from the specified directory, resizes them, flattens them into vectors,
    and assigns labels based on folder names.

    Assumes each class has a subfolder with 'train' images.

    Arguments:
    data_directory -- string, path to the root directory containing class folders
    mode -- string, either 'train' or 'test' to specify which subfolder to load
    image_size -- tuple(int, int), size to which each image will be resized (width, height)

    Returns:
    X -- list of numpy arrays, each array is a flattened image vector
    y -- list of integers, labels corresponding to each image
    label_dict-- dictionary mapping class names (folder names) to integer labels, e.g., {'Dogs': 0, 'Cats': 1}
    """
    X = [] # Empty list for storing flattened image vectors 
    y = [] # Empty list for storing interger labels (0 and 1)
    
    # Get class labels from folder names and map to integers
    
    labels = os.listdir(data_directory) # lists all the sub-folders of the specified root directory, that contain the images
    label_dict = {label: i for i, label in enumerate(labels)}
    
    # Loop through each class folder and creates path for the mode
    for label in labels:
        folder = os.path.join(data_directory, label, mode)
        if not os.path.isdir(folder):
            continue  # Skip if train/test folder doesn't exist

        Files = os.listdir(folder)
        for filename in Files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder, filename)
               # Open image and convert to RGB
                img = Image.open(image_path).convert("RGB")
              # Resizes the loaded image into the size specified above (64x64 pixels)
                img_resized = img.resize(image_size)
              # Creates a numpy array from the resized image
                X_array = np.array(img_resized)
             # Flattens the array into a vector and appends it into X
                X.append(X_array.flatten())
             # Appends each class label into a list "y"
                y.append(label_dict[label])

    # Converts X into an array and normalizes all pixel values to [0,1]
    X_norm = (np.array(X))/255.0
    # Converts y labels into an array
    y_array = np.array(y)
    
    return X_norm, y_array, label_dict
    

In [3]:

# Assigning the outputs of the "loading_images" function to their respective variables
X_train, y_train, label_dict = loading_images("../Images", "train", image_size)
X_test, y_test = loading_images("../Images", "test", image_size)[:2]

# Transpose for math: changing dimensions to (features, training_examples)
X_train, X_test = X_train.T, X_test.T
y_train, y_test = y_train.reshape(1, y_train.shape[0]), y_test.reshape(1, y_test.shape[0])

In [35]:
# ------------------------------------
# 3. PARAMETER INITIALIZATION FUNCTION
# ------------------------------------

def parameter_init(input_size, hidden_1, hidden_2, hidden_3, output_size):
    """Initializes weights and biases for a 4-layer neural network.
    
    Arguments:    
    input_size -- int, number of input features (size of the input layer)
    hidden_1 -- int, number of units in the first hidden layer
    hidden_2 -- int, number of units in the second hidden layer
    hidden_3 -- int, number of units in the third hidden layer
    output_size -- int, number of units in the output layer
    
    Returns:
    parameters -- dict containing initialized weights (W1, W2, W3, W4) 
                  and biases (b1, b2, b3, b4) for each layer
    """
    np.random.seed(1)

    # Weights: initialized with small random values (scaled by 0.01)
    # to prevent exploding activations and help stable learning.
    # Biases: initialized to zero.
    W1 = np.random.randn(hidden_1, input_size)*0.01
    b1 = np.zeros((hidden_1, 1))
    W2 = np.random.randn(hidden_2, hidden_1)*0.01
    b2 = np.zeros((hidden_2,1))
    W3 = np.random.randn(hidden_3, hidden_2)*0.01
    b3 = np.zeros((hidden_3,1))
    W4 = np.random.randn(output_size, hidden_3)*0.01
    b4 = np.zeros((output_size,1))
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2, "W3":W3, "b3":b3, "W4":W4, "b4":b4}
    return parameters
    

# -----------------------------
# 4. ACTIVATION FUNCTIONS
# -----------------------------

def relu(Z):
    A = np.maximum(0,Z)
    return A
def relu_derivative(Z):
    return (Z>0).astype(float)
def sigmoid(Z4):
    y_hat = 1/(1 + np.exp(-Z4))
    return y_hat

def sigmoid_derivative(y_hat):
    y_hat_derivative = y_hat*(1-y_hat)
    return y_hat_derivative

# -------------------------------
# 5. FORWARD BACKWARD PROPAGATION
# -------------------------------

# Before calling this function, make sure to initialize parameters:
# parameters = parameter_init(input_size, hidden_1, hidden_2, hidden_3, output_size)
def forward_prop(X_train, parameters):

    # Retrieve weights and biases from dictionary
    W1, b1 = parameters["W1"], parameters["b1"]
    W2, b2 = parameters["W2"], parameters["b2"]
    W3, b3 = parameters["W3"], parameters["b3"]
    W4, b4 = parameters["W4"], parameters["b4"]

    Z1 = W1.dot(X_train) + b1
    A1 = relu(Z1)  
    Z2 = W2.dot(A1) + b2
    A2 = relu(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = relu(Z3)
    Z4 = W4.dot(A3) + b4
    y_hat = sigmoid (Z4)

    cache = {"Z1":Z1, "A1": A1, "Z2": Z2, "A2": A2, "Z3": Z3,"A3": A3,"Z4": Z4, "A4": y_hat}
    return cache

def Cost_function(y, y_hat):
    m = y.shape[1] # number of training examples
    cost = -np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)) / m
    cost = np.squeeze(cost) 
    return cost
    
# -------------------------------
# 5. BACKWARD BACKWARD PROPAGATION
# -------------------------------

def backward_prob(parameters, cache, X_train, y_train):
    W1, W2, W3, W4 = parameters["W1"], parameters["W2"], parameters["W3"], parameters["W4"]
    A1, A2, A3, A4 = cache["A1"], cache["A2"], cache["A3"], cache["A4"]
    Z1, Z2, Z3, Z4 = cache["Z1"], cache["Z2"], cache["Z3"], cache["Z4"]
    
    m = X_train.shape[1]

    # Layer 4 (output layer)
    dZ4 = A4 - y_train
    dW4 = (1/m) * np.dot(dZ4, A3.T)
    db4 = (1/m) * np.sum(dZ4, axis=1, keepdims=True)

    # Hidden Layer 3
    dZ3 = np.dot(W4.T, dZ4) * relu_derivative(Z3)
    dW3 = (1/m) * np.dot(dZ3, A2.T)
    db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)

    # Hidden Layer 2
    dZ2 = np.dot(W3.T, dZ3) * relu_derivative(Z2)
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    # HiddeLayer 1
    dZ1 = np.dot(W2.T, dZ2) * relu_derivative(Z1)
    dW1 = (1/m) * np.dot(dZ1, X_train.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    derivatives = {
        "dW4": dW4, "db4": db4,
        "dW3": dW3, "db3": db3,
        "dW2": dW2, "db2": db2,
        "dW1": dW1, "db1": db1
    }
    
    return derivatives


In [36]:
parameters=parameters=parameter_init(12288,128,64,32,1)

In [37]:
cache = forward_prop(X_train, parameters)

In [39]:
backward_prob(parameters, cache, X_train, y_train )

{'dW4': array([[ 0.00000000e+00,  5.80425414e-05, -2.59729637e-05,
          4.86582482e-05, -2.77238297e-05,  0.00000000e+00,
         -1.27680301e-06, -1.28750910e-05,  2.14877829e-05,
          2.09960123e-04, -7.55673157e-05,  9.55833450e-07,
          0.00000000e+00, -3.88715108e-05, -4.41348463e-05,
          5.13824009e-05, -6.20678925e-06, -4.39283130e-05,
          7.94671285e-06, -6.44082413e-05,  2.24657363e-05,
         -6.95591758e-06,  8.91982622e-05, -3.82342006e-06,
          1.08519877e-05, -9.80539783e-05, -5.53006787e-05,
          2.32641105e-04,  8.38299507e-05, -8.23396946e-06,
         -4.39787158e-05, -8.80091938e-05]]),
 'db4': array([[1.76234262e-05]]),
 'dW3': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.57590506e-05,  6.48227898e-07, -1.35127565e-06, ...,
          8.95662884e-06, -2.03050844e-06, -1.78870790e-05],
        [-1.18649930e-05,  0.00000000e+00,  8.6686285

In [ ]:
backward_prob()

In [5]:
parameters=parameter_init(12288,128,64,32,1)
y_hat = forward_prop(X_train, parameters)[0]

In [6]:
Cost_function(y_train, y_hat)

0.6931441807084028

In [7]:
y_hat

array([[0.50002217, 0.50000296, 0.5000125 , 0.50001995, 0.50002182,
        0.50001195, 0.50000897, 0.50000907, 0.50001882, 0.50003302,
        0.50003582, 0.50001238, 0.50002145, 0.500018  , 0.50000161,
        0.50000136, 0.50002337, 0.50002958, 0.5000227 , 0.50002496]])

In [8]:
y_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])